# <center> Matrix factorization. Collaborative Filtering</center>

<img src='../week1/images/matrix.png' width=700>

План занятия

- Простые user/item-based similarity подходы и статистические бейзлайны
- Матричные факторизации на примере SVD
- Коллаборативная фильтрация на примере ALS

### Базовые подходы. Item-based и User-based similarity

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import scipy as sp
import scipy.sparse as scs
import matplotlib
import matplotlib.pyplot as plt
from math import sqrt
from numpy.linalg import svd
from sklearn.metrics import r2_score
from tqdm.notebook import tqdm
from PIL import Image
from ipywidgets import interact, widgets

matplotlib.rcParams['figure.figsize'] = (10,9)
np.set_printoptions(2, suppress=True)
plt.rcParams.update({'font.size': 14})
# sns.set_style('whitegrid')

**1. Сделаем первый бейзлайн Top Popular**

Популярность можно посчитать по-разному, например, как наибольшее число просмотров по всем пользователям. Сразу хорошо бы понять про допустимость повторов (например, если речь идет про фильмы). 

In [ ]:
with open('critics_reviews.pickle', 'rb') as f:
    reviews = pickle.load(f)

In [ ]:
def get_pop_movies(reviews):
    movies_counts = {}
    for user, films in reviews.items():
        for film, _ in films.items():
            movies_counts.setdefault(film, 0)
            movies_counts[film] += 1
    movies_counts = dict(sorted(movies_counts.items(), key=lambda x: x[1], reverse=True))
    return movies_counts


def top_pop_recommender(reviews, person):
    recoms = get_pop_movies(reviews)
    recoms = {k: v for k, v in recoms.items() if k not in reviews[person]}
    return recoms

In [ ]:
top_pop_recommender(reviews, 'Toby')

Если у нас много фильмов, какого параметра не хватает для выдачи рекомендаций?  

**2. Top Popular v.2.0.**

Далеко не всегда наиболее часто просматриваемые - это самые лучшие по рейтингу фильмы. Можно посчитать топ фильмов по среднему рейтингу из отзывов пользователей. Сделаем рекомендации по топу популярности на основе средней оценки фильма:

In [ ]:
def get_movies_avg_scores(reviews):
    movies_rating = {}
    for user, films in reviews.items():
        for film, score in films.items():
            movies_rating.setdefault(film, [])
            movies_rating[film].append(score)

    movies_rating = {k: np.mean(v) for k, v in movies_rating.items()}
    movies_rating = dict(sorted(movies_rating.items(), key=lambda x: x[1], reverse=True))
    return movies_rating


def top_avg_scores_recommender(reviews, person):
    recoms = get_movies_avg_scores(reviews)
    new_movies = set(recoms.keys()).difference(reviews[person])
    recoms = {k: round(v, 2) for k, v in recoms.items() if k in new_movies}
    return recoms

top_avg_scores_recommender(reviews, 'Toby')

* К тому же, Top popular - это неперсональные рекомендации. Как вы думаете, если повторы допустимы, какие можно сделать персонализированные бейзлайны? При условии, что есть только интеракции (то есть, никакие дополнительные признаки не доступны).

Из каких шагов будет состоять имплементация Top Personal с повторами в рекомендациях? 


**3. User-based similarity** 

Простые, но надежные бейзлайны можно посчитать в виде рекомендаций на основе сходства пользователей или объекотв (user/item similarity). 

Давайте рассмотрим такой вариант - нужно выдать для $i$-ого пользователя  рекомендации на основе похожих на него пользователей. По его соседям соберем множество просмотренных фильмов и выведем топ  рекомендаций в порядке убывания их среднего рейтинга по соседям. Оценка каждого соседа учитывается с учетом веса его схожести с $i$-м пользователем.

То есть, шаги следующие:

1. Выбор метрики схожести, ее расчет. 
2. Выбор k соседей для формирования множества из их айтемов.
3. Сортировка по убыванию оценок и возвращение top-n рекомендаций пользователю.


Какие простые и распространенные метрики схожести (расстояния) можете вспомнить?

In [ ]:
def sim_distance(reviews, person1, person2):
    sum_of_squares = sum([(reviews[person1][item] - reviews[person2][item])**2
                         for item in reviews[person1] if item in reviews[person2]])
    return 1/(1 + np.sqrt(sum_of_squares))

<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/2b9c2079a3ffc1aacd36201ea0a3fb2460dc226f'>
<img src='https://wikimedia.org/api/rest_v1/media/math/render/svg/b87fab4bd95646a6aa894efe96e894761c94498f'>

где $n$ - размер выборки, <br>
$x_{i},y_{i}$ - оценки первого и второго пользователя <br>
$\bar x = \frac{1}{n} \sum^n_{i=1}x_i$

In [ ]:
def sim_pearson(reviews, person1, person2):
    stats = {}
    similar = list(set(reviews[person1].keys()) & set(reviews[person2].keys()))
    for person in [person1, person2]:
        stats.setdefault(person, {'ranks': 0})
        stats[person]['ranks'] = [reviews[person][i] for i in similar]
        
    return np.corrcoef(stats[person1]['ranks'], stats[person2]['ranks'])[0, 1]

Для наглядности выведем попарные метрики схожести пользователей:

In [ ]:
compared = set()
for first in reviews:
    for second in reviews:
        compared.add((first, second))
        if first != second and (second, first) not in compared:
            print(f'{first} & {second}: ', round(sim_pearson(reviews, first, second), 2))

Давайте посмотрим на коэффициенты корреляции. У нас нашлось несколько примеров, у которых коэффициент корреляции равен 1. И есть даже сильная обратная линейная взаимосвязь: -1.

Какие есть недостатки при использовании этой метрики? Как можно использовать пары с отрицательными корреляциями?

Сортировка самих пользователей на основе схожести:

In [ ]:
def sort_users(reviews, person, n=5, similarity=sim_pearson):
    scores = [(other, round(similarity(reviews, person, other),2)) for other in reviews if other != person]
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return scores[:n]

In [ ]:
sort_users(reviews, 'Toby', n=6)

Перейдем к самим рекомендациям по найденным похожим пользователям.

Кстати, сколько выбирать соседей?

По всем похожим пользователям (sim>0) берем их рейтинги по всем новым для таргетного пользователя фильмам. Считаем взвешенную сумму эти рейтингов на скор схожести соседа и нормализуем, поделив на сумму скоров схожести всех соседей. 

In [ ]:
def get_recoms_by_users(prefs, person, similarity=sim_pearson):
    totals = {}
    sim_sums = {}
    for other in prefs:
        if other == person:
            continue
        sim = similarity(prefs, person, other)
        if sim <= 0:
            continue
        for item in prefs[other]:
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item, 0)
                # print(person, other, sim, item, prefs[other][item])
                totals[item] += prefs[other][item]*sim
                sim_sums.setdefault(item, 0)
                sim_sums[item] += sim
    rankings = [(item, round(total/sim_sums[item], 2)) for item, total in totals.items()]
    rankings = sorted(rankings, key=lambda x: x[1], reverse=True)
    
    return rankings

In [ ]:
get_recoms_by_users(reviews, 'Toby', sim_distance)

**Item-based similarity**

Можно посчитать все то же самое, но получить рекомендации пользователя на основе похожести просмотренных им ранее фильмов.

| Метод | Достоинства  |  Недостатки | 
|---|---|---|
| User/item-based similarity | Простота подходов и имплементации|  Для расчетов нужны оценки пользователей (explicit)|  
|
|| Общая интерпретируемость модели (на основе предпочтений похожих пользователей) | Невозможность предсказаний при холодном старте* |  
|
|| Хорошее качество для бейзлайна при не сильно разреженных данных | Холодный user/item сам никогда не попадет в рекомендации | 
|
|| Вычислительно незатратно| Ненадеждность корреляций |  
|| Можно быстро делать обновления (не зависим от всей выборки)| Popularity bias, поэтому нужно взвешивание, нормализация| 
|

\* **Cold start problem (проблема холодного старта)** - проблема отсутствия данных по интеракциям у пользователей или айтемов. Неустойчивые к этой проблеме модели тренируются на обучающей выборке и не могут сделать предсказания для новых пользователей и/или айтемов, которые не были включены в train. Например, out-of-sample наблюдения или просто появились новые пользователи/айтемы. 

### Что такое колаборативная фильтрация и MF

**Коллаборативная фильтрация**(CF, Collaborative Filtering) одновременно использует сходство между пользователями и айтемами для рекомендаций. В результате алгоритма мы получает латентные эмбеддинги по пользователям и айтемам исходя из данных матрицы интеракций, в том числе и на implicit данных.

Будем исходит из гипотезы, что чем более похожи/релевантны пара $(user_i, item_j)$, тем больше должно быть значение их скалярного произведения эмбеддингов.

**Матричная факторизация** (Matrix Factorization, MF) - это матричное разложение, которое по сути является простой латентной эмбеддинг моделью. Будем раскладывать исходную матрицу $A$ на несколько матриц (обычно их 2 или 3), которые будут ее аппроксимировать при перемножении $ PQ^T$. Таким образом:

* Матрица интеракций $A \in R^{m \times n} $ <br>
* Матрица эмбеддингов пользователей $P \in R^{m \times dim}$, где ряд $i$ - это эмбеддинг пользователя $i$. <br>
* Матрица эмбеддингов айтемов $Q \in R^{n \times dim}$, где ряд $j$ - это эмбеддинг айтема $j$.

Пара $(i, j)$ в $PQ^T$ - это просто скалярное произведение $<P_i, Q_j>$ эмбедднигов пользователя i и айтема j, которое должно быть близко к $A_{ij}$

**Определение целевой функции для MF**

Одной из интуитивно понятных целевых функций для MF explicit данных является квадрат расстояния. Минимизируем по всем парам наблюдаемых записей: 

$$min_{P \in R^{m\times d}, Q \in R^{n\times d}} \sum_{(i,j) \in obs} (A_{ij} - <P_i, Q_j>)^2$$

Вы можете решать эту квадратичную задачу с помощью сингулярного разложения матрицы (SVD). Однако SVD тоже не лучшее решение, потому что на практике матрица может быть очень разреженной. Например, подумайте сколько может быть просмотрено конкретным пользователем видео на YouTube из всего их множества. Решение (соответствующее аппроксимации входной матрицы моделью), скорее всего, будет близко к нулю, что тоже приведет к плохой обобщающей способности.

**Что же тогда делать?** См. доп. материал в конце ноутбука

In [ ]:
A = np.random.random((20, 5))*5
A

In [ ]:
U, s, V = svd(A)
U.shape, s.shape, V.shape

In [ ]:
S = np.zeros(A.shape)
S[:len(s), :len(s)] = np.diag(s)
S

In [ ]:
U @ S @ V

In [ ]:
np.allclose(A, U @ S @ V)

In [ ]:
def reshape_for_k(k: int, U, s, V):
    U_new = U[: , :k]
    S = np.zeros((U.shape[0], V.shape[1]))
    S[:len(s), :len(s)] = np.diag(s)
    S_new = S[: k, : k]
    V_new = V[: k, :]
    return U_new @ S_new @ V_new

reshape_for_k(2, U, s, V)

**Сколько компонент брать?**

Мы не хотим получить overfit и underfit.

Можно построить простые графики метрики качества задачи восстановления исходных данных. Например, **MSE** или **MAE**.

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/2021/week2/images/svd3.png' width=500>

На третьей компоненте и до пятой величина MSE практически не меняется, по этому остановимся на трёх компонентах.

Мы можем добавить коэффициент объясненной дисперсии и разреженность.


**Explained variance ratio (EVR)** — коэффициент объясненной дисперсии (оценки информативности компонентов). Эта метрика позволяет вам понять, какой процент общей дисперсии данных объясняется каждой из компонент. Считается так - для каждой компоненты нужно разделить ее собственное или сингулярное значение на сумму всех собственных или сингулярных значений.

**Sparsity** — разреженность данных, оценить которую можно отношением количества элементов больше 0 к количеству всех элементов.

Как бы вы проинтерпретировали этот график?

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week2/images/svd2.png' height=700 width=500>

Какие еще факторы важны?

1. Качество самих рекомендаций (метрики ранжирования на оффлайн и онлайн) данных
2. Вычислительные ресурсы (RAM, жетский диск, требования к скорости расчетов и т.д.)

Давайте сначала вспомним и повизуализируем, что значит выбор компонент из SVD на примере картинок и как это влияет на качество восстановления на практике:

In [ ]:
nice_pic = np.array(Image.open('../week1/images/neo.png').convert('L'))
plt.imshow(nice_pic, cmap='gray');

In [ ]:
U, s, V = svd(nice_pic)
print((U.shape, s.shape, V.shape))

In [ ]:
S = np.zeros(nice_pic.shape)
S[:len(s), :len(s)] = np.diag(s)
print(S.shape)
S

In [ ]:
plt.imshow(U @ S @ V, cmap='gray');

In [ ]:
def reshape_for_k(k: int, U, s, V):
    U_new = U[: , :k]
    S = np.zeros((U.shape[0], V.shape[1]))
    S[:len(s), :len(s)] = np.diag(s)
    S_new = S[: k, : k]
    V_new = V[: k, :]
    return U_new @ S_new @ V_new

In [ ]:
new_img = reshape_for_k(15, U, s, V)
plt.imshow(new_img, cmap='gray');

In [ ]:
@interact(k=widgets.IntSlider(min=1,max=631,step=10,value=10))
def plot_figure(k):
    plt.title(f'k = {k}')
    new_img = reshape_for_k(k, U, s, V)
    plt.imshow(new_img, cmap='gray')

Попробуем применить SVD на табличных данных для задачи рекомендации

In [ ]:
ratings = pd.read_csv('ratings.csv')
ratings

In [ ]:
num_users, num_movies = ratings.userId.nunique(), ratings.movieId.nunique()
num_users, num_movies

Разделим выборку на обучение и тест следующим образом: для каждого пользователя в тестовую выборку попадут 10 его последних оценок.

In [ ]:
sns.displot(train_ratings.rating);

In [ ]:
sns.displot(train_ratings.groupby('movieId').size(), log_scale=True, height=7);

In [ ]:
train_ratings, test_ratings = [], []
num_test_samples = 10

for userId, user_data in ratings.groupby('userId'):
    train_ratings += [user_data[:-num_test_samples]]
    test_ratings += [user_data[-num_test_samples:]]

train_ratings = pd.concat(train_ratings)
test_ratings = pd.concat(test_ratings)
train_ratings.shape, test_ratings.shape

In [ ]:
R = scs.coo_array((train_ratings.rating, (train_ratings.userId, train_ratings.movieId)), 
                       shape=(num_users, num_movies)).tocsr()
matrix = R.toarray()

In [ ]:
U, s, V = svd(matrix)
U.shape, s.shape, V.shape

In [ ]:
S = np.zeros(matrix.shape)
S[:len(s), :len(s)] = np.diag(s)
print(S.shape)
S

In [ ]:
preds_matrix = U @ S @ V

In [ ]:
np.allclose(matrix, preds_matrix)

In [ ]:
k = 20
df = pd.DataFrame(data=(zip(matrix[0, :], preds_matrix[0, :])), 
                  columns=['init', 'preds']).sort_values(by='init', ascending=False)
df.sort_values(by='preds', ascending=False).head(k)

<ins>Вопросы и замечания:</ins> 

*  1. В чем может быть практическая польза хранить не исходную матрицу, а две/три новые матрицы? 

*  2. Что важнее - сама оценка скалярного произведения или порядок между оценками?

* 3. Подойдет ли нам наиболее точная аппроксимация исходной матрицы для рекомендательных систем (т.е. полностью точно воспроизвели все значения в материце интеракций)? Почему? 

In [ ]:
# напишите метрики качества ранжирования и посчитайте на тестовых данных, насколько хороша SVD для рекомендаций 
# YOUR CODE HERE

### ALS

Итак, поставлена задача построения модели со скрытыми переменными (latent factor model) для коллаборативной фильтрации:

$$\sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \to \min_{P,Q}.
\label{LFM}$$

Напомним, что суммирование ведется по всем парам $(u, i),$ для которых известен рейтинг $r_{ui}$ (и только по ним), а $p_u, q_i$ – латентные представления пользователя $u$ и товара $i$ из соответствующих матрицы $P, Q$.

Подход ALS (Alternating Least Squares) решает задачу, попеременно фиксируя матрицы $P$ и $Q$, — оказывается, что, зафиксировав одну из матриц, можно выписать аналитическое решение задачи для другой.

$$\nabla_{p_u} \bigg[ \sum_{u,i} (r_{ui} - \langle p_u, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)q_i = 0$$

Воспользовавшись тем, что $a^Tbc = cb^Ta$, получим
$$\sum_{i} r_{ui}q_i - \sum_i q_i q_i^T p_u = 0.$$
Тогда окончательно каждый столбец матрицы $P$ можно найти по формуле
$$p_u = \bigg( \sum_i q_i q_i^T\bigg)^{-1}\sum_ir_{ui}q_i \;\; \forall u,$$
аналогично для столбцов матрицы $Q$
$$q_i = \bigg( \sum_u p_u p_u^T\bigg)^{-1}\sum_ur_{ui}p_u \;\; \forall i.$$

Таким образом мы можем решать оптимизационную задачу, поочередно фиксируя одну из матриц $P$ или $Q$ и проводя оптимизацию по второй.

**Оригинальная статья (для explicit feedback):**

Bell, R.M. and Koren, Y., 2007, October. Scalable collaborative filtering with jointly derived neighborhood interpolation weights. In Seventh IEEE international conference on data mining (ICDM 2007) (pp. 43-52). IEEE.


### iALS


**Статья для implicit данных:**


Implicit feedback ALS:
* Hu, Y., Koren, Y. and Volinsky, C., 2008, December. Collaborative filtering for implicit feedback datasets. In 2008 Eighth IEEE international conference on data mining (pp. 263-272). Ieee. http://yifanhu.net/PUB/cf.pdf

**Особенности:**

1. Нет explicit данных (явных позитивных и отрицательных оценок). <br>
2. Много шума в данных.  <br>
3. Используются свои функции и метрики для оценки implicit feedback. Если в explicit $r_ui$ - это оценки предпочтений пользователя, то в implicit - это степень уверенности. <br>


| Метод | Достоинства  |  Недостатки | 
|---|---|---|
| Collaborative Filtering | Для предсказаний достаточно одних интеракций |  Страдает от проблемы холодного старта|  
|
|| Serendipity (может выдать для пользователя что-то неожиданно релевантное, создавая интерес к новому) | Низкое качество рекомендаций при маленьком датасете или слишком разреженной матрице |  
|
|| Общая интерпретируемость модели (на основе предпочтений похожих пользователей) | Не предполагает использование признаков| 
|
||  | User/item никогда не попадет в рекомендации, если нет интеракций|  
||  | Popularity bias, поэтому нужно взвешивание, нормализация| 
|


Наша задача оптимизации в том, что мы стремимся минимизировать целевую функцию и найти
оптимальные $X$ и $Y$. В частности, мы стремимся минимизировать ошибку наименьших квадратов наблюдаемых оценок
(и упорядочить):

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week2/images/als_loss.png' width=700 height=500>

Для implicit feedback:

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week2/images/als_imp.png' width=600 height=600>
<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week2/images/als_sol.png' width=500 height=400>


Авторы статьи вводят набор переменных, $c_ui$, которые измеряют нашу уверенность в наблюдении за $p_ui$. Чем больше значение наблюдения за пользовательским элементом, тем больше вы уверены в этом значении. Таким образом, у нас есть некоторая минимальная уверенность в $p_ui$ для каждой пары пользовательских элементов, но по мере того, как мы наблюдаем больше положительных предпочтений по паре пользователь-объект, наша уверенность в $p_ui$ = 1 соответственно возрастает. Скорость увеличения регулируется константой $\alpha$.

<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week2/images/als.png' width=700 height=500>


На таких маленьких данных мы можем позволить себе обучить полный ALS с честным обращением матриц.

In [ ]:
def ALS(user_ids, item_ids, ratings, num_users, num_items, num_dims=32, num_iters=10, eps=1e-7):
    R = scs.coo_array((ratings, (user_ids, item_ids)), shape=(num_users, num_items)).tocsr()
    X = np.random.randn(num_users, num_dims) #U
    Y = np.random.randn(num_items, num_dims) #V
    
    for t in tqdm(range(num_iters)):
        RY = R @ Y
        for u in range(num_users):
            relevant_items = item_ids[user_ids == u]
            Y_rel = Y[relevant_items]
            YY = Y_rel.reshape(-1, num_dims, 1) * Y_rel.reshape(-1, 1, num_dims)
            X[u] = np.linalg.inv(YY.sum(axis=0) + eps * np.eye(num_dims)) @ RY[u]

        RX = R.T @ X
        for i in range(num_items):
            relevant_users = user_ids[item_ids == i]
            X_rel = X[relevant_users]
            XX = X_rel.reshape(-1, num_dims, 1) * X_rel.reshape(-1, 1, num_dims)
            Y[i] = np.linalg.inv(XX.sum(axis=0) + eps * np.eye(num_dims)) @ RX[i]
    
    return R, X, Y

R, X, Y = ALS(train_ratings.userId, train_ratings.movieId, train_ratings.rating, num_users, num_movies, num_iters=10)

In [ ]:
num_users, num_movies

Сгенерируем предсказания, посчитав скалярные произведения между векторами пользователей и фильмов.

In [ ]:
train_preds = (X[train_ratings.userId] * Y[train_ratings.movieId]).sum(axis=1)
test_preds = (X[test_ratings.userId] * Y[test_ratings.movieId]).sum(axis=1)
test_preds = np.clip(test_preds, 0.5, 5.0)
print(f'Train R^2: {r2_score(train_ratings.rating, train_preds):.3f}')
print(f'Test R^2: {r2_score(test_ratings.rating, test_preds):.3f}')

Кстати, в каких случаях $R^2$ получается отрицательным?

Результаты получились весьма удручающими. Но не будем забывать, что нас интересует рекомендация фильмов, а не предсказание точно рейтинга. Поэтому упорядочим фильмы по предсказанной релевантности (значение скалярного произведения) и посчитаем метрику для ранжирования — MAP@k, где пусть $k=10$. Для сравнения рассмотрим еще два алгоритма — случайный (ранжирует фильмы в произвольном порядке) и алгоритм, возвращающий топ рейтинга фильмов.

In [ ]:
test_relevant = []
for user_id, user_data in test_ratings[test_ratings.rating >= 4.0].groupby('userId'):
    test_relevant += [user_data.movieId.tolist()]

In [ ]:
def remove_train_items(preds, k):
    new_preds = np.zeros((preds.shape[0], k), dtype=np.int)
    for user_id, user_data in train_ratings.groupby('userId'):
        user_preds = preds[user_id]
        new_preds[user_id] = user_preds[~np.in1d(user_preds, user_data.movieId)][:k]
    
    return new_preds


def ALS_prediction(X, Y, k=10):
    preds = np.argsort(X @ Y.T, axis=1)
    return preds[:k]


def random_prediction(k=10):
    preds = np.tile(np.arange(num_movies), (num_users, 1))
    for i in range(num_users):
        rand_perm = np.random.permutation(num_movies)
        preds[i] = preds[i][rand_perm]

    return preds[:k]

def top_prediction(freq_thr=10, k=10):
    mean_rating = train_ratings.groupby('movieId').rating.mean()
    mean_rating = mean_rating[train_ratings.groupby('movieId').size() >= freq_thr]
    preds = np.array(mean_rating.sort_values(ascending=False).index)
    preds = np.tile(preds, (num_users, 1))
    return preds[:k]


def MAPk(y_true, y_pred, k=10):
    map_k, count = 0, 0
    for relevant_items, predicted_items in zip(y_true, y_pred):
        if not relevant_items:
            continue
        correct = np.isin(predicted_items[:k], relevant_items)
        map_k += (correct / np.arange(1, k + 1)).sum() / \
            (1 / np.arange(1, len(relevant_items) + 1)).sum()
        count += 1
    
    map_k = map_k / count
    return map_k

Посчитаем значение MAP@k для разных значений k.

In [ ]:
ks = np.arange(1, 51)
als_preds = ALS_prediction(X, Y, k=ks[-1])
random_preds = random_prediction(k=ks[-1])
top_preds = top_prediction(freq_thr=10, k=ks[-1])

random_mapk = [MAPk(test_relevant, random_preds, k=k) for k in ks]
top_mapk = [MAPk(test_relevant, top_preds, k=k) for k in ks]
als_mapk = [MAPk(test_relevant, als_preds, k=k) for k in ks]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ks, random_mapk, label='Random')
plt.plot(ks, top_mapk, label='Top rating')
plt.plot(ks, als_mapk, label='ALS')

plt.legend(title='method')
plt.xlabel('k')
plt.ylabel('MAP@k')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(ks, random_mapk, label='Random')
plt.plot(ks, top_mapk, label='Top rating')
plt.plot(ks, als_mapk, label='ALS')

plt.legend(title='method')
plt.xlabel('k')
plt.ylabel('MAP@k')
plt.show()

Померяем также две метрики, которые характеризуют разнообразность предсказаний: coverage — доля фильмов из базы, которые в принципе рекомендуются и personalization — среднее косинусное расстояние между бинарными векторами рекомендаций для разных пользователей.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def coverage(preds):
    return np.unique(preds.flatten()).size / num_movies


def personalization(preds):
    N = preds.shape[0]
    ohe_matrix = np.zeros((N, num_movies))
    ohe_matrix[np.arange(N).repeat(preds.shape[1]), preds.reshape(-1)] = 1.
    sim_matrix = cosine_similarity(ohe_matrix)
    return 1 - (sim_matrix.sum() - N) / (N * (N - 1))

print(f'Random coverage: {coverage(random_preds):.4f}')
print(f'Top rating coverage: {coverage(top_preds):.4f}')
print(f'ALS coverage: {coverage(als_preds):.4f}')

Кроме того, можем попробовать использовать обученные векторы фильмов, чтобы выдавать фильмы, похожие на данные запрос, например:

In [ ]:
example_movieId = ratings[ratings.title=='Iron Man'].iloc[0].movieId
scores = Y @ Y[example_movieId]
similar_movies = np.argsort(-scores)[:20]

for movieId in similar_movies:
    normalized_score = scores[movieId] / np.linalg.norm(V[example_movieId]) / np.linalg.norm(V[movieId])
    print(f'{ratings[ratings.movieId == movieId].iloc[0].title}: {normalized_score:.3f}')


| Критерий | ALS  |  iALS | 
|---|---|---|
| Тип данных | Этот подход используется, когда у вас есть явные рейтинги или оценки, предоставленные пользователями для различных предметов. Например, пользователи могут оценивать фильмы по шкале от 1 до 5. |  Этот подход используется, когда у вас есть неявные сигналы взаимодействия между пользователями и предметами, такие как клики, просмотры, покупки или временные интервалы между действиями. |  
|
|Цель | Модель стремится точно предсказать явные рейтинги или оценки, которые пользователи могли бы дать предметам. | Модель стремится моделировать уровень уверенности или важности взаимодействия между пользователем и предметом, но не предсказывает явные рейтинги.|  
|
|Функции потерь| Обычно использует функцию потерь, такую как среднеквадратичная ошибка (MSE), для минимизации разницы между предсказанными и фактическими оценками. | Использует функцию потерь, которая учитывает уверенность в неявных взаимодействиях и стремится увеличить уверенность для более важных взаимодействий.| 
|
|Взвешивание| Не уделяет внимания взаимодействиям, которые не были явно оценены пользователями. Исключает информацию о неявных действиях. | Дополнительная параметризация. Учитывает все неявные действия, но с учетом их уверенности или веса, что позволяет модели учесть важность различных видов взаимодействий| 
|

### Доп. материалы

Есть **Weighted Matrix Factorization** (взвешенная матричная факторизация), которая раскладывает целевую функцию на две суммы:

1) сумма по всем имеющимся интеракциям

2) сумма по отсутствующим интеракциям (которые мы обозначали 0)


$$min_{P \in R^{m\times d}, Q \in R^{n\times d}} \sum_{(i,j) \in obs} (A_{ij} - <P_i, Q_j>)^2 + w_{0} \cdot \sum_{(i,j) \notin obs} (<P_{i}, Q_{j}>)^2$$

Здесь $w_{0}$ — гиперпараметр, который взвешивает две компонентны, чтобы одна не преобладала над другой. Настройка этого гиперпараметра очень важна.

На практике, вам также необходимо взвешивать наблюдаемые пары. Например, частые элементы (очень популярные видео на YouTube) или слишком активные пользователи могут доминировать в целевой функции. Вы можете скорректировать этот эффект, взвесив обучающие примеры, чтобы учесть их частоту в датасете. Можно заменить целевую функцию на:

$$min_{P \in R^{m\times d}, Q \in R^{n\times d}} \sum_{(i,j) \in obs} w_{i,j} \cdot (A_{ij} - <P_i, Q_j>)^2 + w_{0} \cdot \sum_{(i,j) \notin obs} (<P_{i}, Q_{j}>)^2$$


**Минимизация целевой функции**:

* SGD (stochastic gradient descent) - общеизвестный и используемый везде метод, оптимизация происходит по наблюдениям из обучающей выборки. 
* MCMC (Markov chain Monte Carlo) - сэмплирование параметров из апостериорного распределения. 
* Weighted Alternating Least Squares (WALS) - специализированный лос под обозначенную выше целевую функцию, с попеременной оптимизацией по каждому параметру при фиксированных значениях остальных параметров. 

Лосс квадратичный для обеих матриц $P$ и $Q$.  
WALS случайным образом инициализирует эмбеддинги и потом последовательно обновляет веса то одной, то второй матрицы:

1) Фиксируем $P$, делаем итерацию оптимизации для решения $Q$. 

2) Фиксируем $Q$, делаем итерацию оптимизации для решения $P$. 

Каждый этап может быть решен в явном виде (через решение линейной системы). Этот метод гарантированно сходится, потому что каждый шаг гарантированно уменьшает потери.



| Метод оптимизации  | Достоинства  |  Недостатки | 
|---|---|---|
|  SGD |  Гибкий в использовании, можно брать другие лоссы | Медленнее, небыстро сходится  |  
|  |  Можно распараллелить вычисления | Сложнее работать с отрицательными примерами, нужен negative sampling  |  
|  WALS |  Быстрее сходится, чем SGD | Применим только к квадратичным лоссам  | 
|  |  Легче обрабатывать отсутсвующие интеракции |  |
|   |  Можно распараллелить вычисления|  | 



## SVD 

<img src='https://www.dataminingapps.com/wp-content/uploads/2020/02/svd2.png'>

SVD, помимо скалярного произведения, включает еще смещения по пользователю $b_u$ и объекту $b_i$. Предполагается, что смещения пользователей отражают тенденцию некоторых пользователей оценивать товары выше (или ниже) среднего. То же самое касается товаров: некоторые товары обычно оцениваются выше, чем некоторые другие. Модель SVD тогда выглядит следующим образом:

$$\hat r_{u,i} = \mu + b_u + b_i + q_{i}^{T}p_{u},$$

где $\mu$ - глобальное среднее по всем оценкам в данных. Тогда задача оптимизации сводится к:

$$ \sum(r_{u,i} - \hat r_{u,i}) ^ 2 +     \lambda(b_i^2 + b_u^2 + ||q_i||^2 + ||p_u||^2),$$

где $\lambda$ - параметр регуляризации.

Оптимизация через стохастический градиентный спуск:

$$b_u \leftarrow b_u + \gamma (e_{ui} - \lambda b_u)$$
$$b_i \leftarrow b_i + \gamma (e_{ui} - \lambda b_i)$$  
$$p_u \leftarrow p_u + \gamma (e_{ui} \cdot q_i - \lambda p_u)$$
$$q_i \leftarrow q_i + \gamma (e_{ui} \cdot p_u - \lambda q_i),$$

где $\gamma$ - это learning rate, <br>
$e_{ui} =  r_{ui} - \hat r_{u,i} = r_{u,i} - (\mu + b_u + b_i + q_{i}^{T}p_{u})$ ошибка алгоритма на паре $(u, i)$.

### Hierarchical Alternating Least Squares

Подход ALS требует вычислений обратных матриц, что накладывает сильные ограничения на размерность скрытых переменных. Альтернативой ALS является подход HALS (Hierarchical Alternating Least Squares), в котором на каждой итерации мы решаем оптимизационную задачу относительно только одной строчки матрицы $P$ или $Q$. Выведем аналитические формулы для $k$-ой строчки матрицы $P$.

$$\frac{\partial}{\partial P_{ku}} \bigg[ \sum_{u',i} (r_{u'i} - \langle p_{u'}, q_i \rangle)^2 \bigg] = \sum_{i} 2(r_{ui} - \langle p_u, q_i \rangle)Q_{ki} = 0$$

$$\sum_{i} (r_{ui} - \sum_{s\neq k} P_{su} Q_{si} )Q_{ki} - P_{ku}\sum_i Q_{ki}^2 = 0$$

$$P_{ku} = \frac{\sum_{i} (r_{ui} - \sum_{s\neq k} P_{su} Q_{si} )Q_{ki}}{\sum_i Q_{ki}^2}$$

### Neural Collaborative Filtering

<img src='https://images.deepai.org/converted-papers/2004.12212/images/NCF.png' width=500>

Нелинейным обобщением матричных разложений является коллаборативная фильтрация с помощью нейронных сетей. Сопоставим каждому пользователю $u$ one-hot encode вектор $\alpha_u$, у которого на $u$-ом месте стоит $1$, а остальные координаты заполнены нулями, аналогично определим вектор $\beta_i$ для товара $i$. В качестве алгоритма классификации (или регрессии, в зависимости от задачи) будем использовать нейросеть, у которой два полносвязных слоя на входе: один соответствует пользователям и принимает на вход вектор $\alpha_u$, а второй соответствует товарам и принимает на вход вектор $\beta_i$. После входных полносвязных слоев соответствующие представления необходимо сконкатенировать и передать в следующие полносвязные слои. На выходе нейронной сети мы получаем предсказание $\widehat{r}_{ui}$ (на рисунке обозначено как $\widehat{y}_{ui}$), которое сравниваем с истинным ответом для заданной пары $r_{ui}$ (на рисунке обозначен как $y_{ui}$).

Несмотря на свою простоту модель обладает рядом важных достоинств:

* мы можем легко обобщить модель на случай, когда у нас есть признаки пользователей или товаров, добавив эти признаки к векторам $\alpha_u$ и $\beta_i$ соответственно (также заметим, что добавление признаков в модель позволяет частично решить проблему холодного старта);
* в зависимости от размеров выборки и природы данных мы можем адаптировать архитектуру под свои нужды, например, если у нас есть картинки в качестве признаков товаров, мы можем добавить сверточные слои в нашу нейронную сеть и обучать всю модель end-to-end.

В этой модели мы по-прежнему можем получить латентные представления пользователей и товаров. На самом деле, латентными представлениями будут столбцы матрицы весов в первых полносвязных слоях (в предположении, что мы умножаем матрицу весов на вектор признаков справа).


Советую посмотреть, как считается ALS при работе с дистрибутивными вычислениями (в зависимости от метода: join, broadcast). Так же, есть стриминг ALS. Это когда мы предполагаем, что данные рейтингов $r_ui$ передаются потоком, и что матрицы коэффициентов $X$, $Y$ помещаются в память каждой машины. Мы предполагаем, что поток смешан (shuffled) и мы можем использовать стохастический градиентный спуск(SGD)
чтобы обновить матрицы коэффициентов X, Y. 
<img src='https://raw.githubusercontent.com/anamarina/RecSys_course/main/week2/images/als_st.png' width=500 height=500>

1. https://developers.google.com/machine-learning/recommendation/collaborative/summary 
2. https://github.com/esokolov/ml-course-hse/blob/master/2021-spring/seminars/sem24-recommendations.pdf
3. https://github.com/esokolov/ml-course-hse/blob/master/2021-spring/seminars/sem24-recommendations.ipynb
4. https://github.com/esokolov/ml-course-hse/blob/master/2021-spring/homeworks-practice/homework-practice-13-recommendations/homework-practice-13-recommendations.ipynb
